## PHYS 105A:  Introduction to Scientific Computing

# Random Numbers and Monte Carlo Methods

Chi-kwan Chan

* In physical science, students very often start with the concept that everything can be done exactly and deterministically.

* This can be one of the biggest misconcept!

* Many physical processes are non-deterministic by nature.  Examples include:
  * Radioactive decay
  * Quantum mechanics

* Sometime, even when the governing equaitons are deterministic, the results are still non-deterministic.  Examples include:
  * 3-body problem
  * Brownian motion
  * Thermal noise

* Therefore, in computer we need some way to model these non-deterministic systems.

* For electronic computers, all operations are deterministic.

* Nevertheless, we can approximate random processes by *pseudo* random number generators.

* These pseudo random number generators can then be used to model non-deterministic systems.

* In addition, people started to realize, even for deterministic problems, randomizing them can still be a very powerful numerical method!  Applications include
  * Numerical integration of high-dimensional space
  * Statistical inference

* This results a large number of random number based numerical methods.

* Monte Carlo is an area of Monaco well known for its world-famous Place du Casino.

* Monte Carlo methods are used to refer to random number based numerical methods.

![Monte Carlo](https://upload.wikimedia.org/wikipedia/commons/1/1b/Monaco_pano.jpg)

In [ ]:
# In order to understand the concept of a random number generator, let's implement one ourself.

mynext = 1

def myrand(): # NOT RECOMMENDED for real application.
    global mynext
    mynext = mynext * 1103515245 + 12345
    return (mynext//65536) % 32768

# This random number generator would generate integers in the domain [0, 32768).
# This random is usually provided to user by

MYRAND_MAX = 32768-1

# There are reasons for choosing the strange constants.  Take a look at
# https://en.wikipedia.org/wiki/Linear_congruential_generator
# if you are interested.

In [ ]:
# Now, every time we run `rand()`, we will get a different number

myrand()

In [ ]:
# For we may just print many of them at the same time:

Rs = [myrand() for i in range(100)]
print(Rs)

In [ ]:
# We may even plot the random numbers

from matplotlib import pyplot as plt

plt.imshow([[myrand() for i in range(100)] for j in range(100)])

* The above random number generator is very simple and is the *sample* implementation in many ANSI C libraries!

* However, because how the standard was written, this create a lot problems.
  * The standard only require RAND_MAX be at least 32767.  If one want to evulate 1e6 points (which is pretty small, as we will see below), you will actually be evaluating the same 32768 points 30 times each!
  * Some implementation "tried" to imporve the algorithm, e.g., swapping the lower and higher bytes.  But these tricks sometime ruins the generator!
  * We mentioned that integrating high-dimension space is an important application of Monte Carlo methods.  However, the above random number generator create correlation in k-space.
  
* Thankfully, `ptyhon`'s random number generator is based on the better [Mersenne Twister algorithm](https://en.wikipedia.org/wiki/Mersenne_Twister).

* From now on, unless for demostration purpose, we will use python's built-in random number generators.